In [5]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('team10_2d.vol')
# mesh.Curve(2)
# mesh.Refine()

In [6]:
from bhdata import BHCurves
fun_dw  = BHCurves(3)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

lin = 0

BH data from TEAM 20 problem
returning energy


In [7]:
%run J_2D.ipynb

Using 2D mesh with ne= 52064 elements and nv= 26337 points and  78400 edges and  626593 DOFs. Actual DOFs after condensation: 418337 .
 


In [8]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

rot = ngs.CF((0,1,-1,0), dims=(2,2))
def curl2d(a):
    return rot*ngs.grad(a)

# H1 = ngs.H1(mesh, order = deg)
H1 = ngs.H1(mesh, order = deg, dirichlet = "ambient_face")
u,v = H1.TnT()

# Nonlinear:

maxit = 100000
tol2 = 1e-8
regb = 1e-12

A = ngs.GridFunction(H1)
B = curl2d(A)
normB = ngs.sqrt(B*B + regb)

ir = ngs.IntegrationRule(ngs.fem.ET.TRIG, order = 2*deg)


if lin == 1: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: nu0/2*B*B}, default = nu0/2*B*B).Compile()
else: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - curl2d(A)*Hs, mesh, order = 2*deg)
    # with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(Hs)*A, mesh)
    return res


if lin == 1: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: nu0}, default = nu0).Compile()
else: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

# rhs = ngs.LinearForm((cf_rhs*B*curl2d(v) - curl2d(v)*Hs)*ngs.dx)
rhs = ngs.LinearForm(H1)
rhs += ngs.SymbolicLFI(cf_rhs*B*curl2d(v) - ngs.curl(Hs)*v, intrule = ir)
# rhs = ngs.LinearForm((cf_rhs*B*curl2d(v) - ngs.curl(Hs)*v)*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,
             0,1), dims=(2,2))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1],
              B[1]*B[0], B[1]*B[1]), dims = (2,2))


fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

if lin == 1: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: nu0*Id}, default = nu0*Id).Compile()
else: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

# ir = ngs.IntegrationRule(points = [(1/3,1/3)], weights = [1/2])
K_iter = ngs.BilinearForm(H1)
# K_iter += ngs.SymbolicBFI(cf_iter*curl2d(u)*curl2d(v))
K_iter += ngs.SymbolicBFI(cf_iter*curl2d(u)*curl2d(v), intrule = ir)
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [9]:
print("Using 2D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,H1.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0)))

du = ngs.GridFunction(H1)
uo = ngs.GridFunction(H1)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(H1.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = H1.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    with ngs.TaskManager():
        du.vec.data = da.mat.Inverse(H1.FreeDofs(), inverse = "sparsecholesky") * dw.vec
        iterativeSolver = CGSolver(K_iter.mat, freedofs = H1.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
        # iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-8,  maxiter = maxit)
        # du.vec.data = iterativeSolver * dw.vec
    
    if len(iterativeSolver.residuals) == maxit: print("... Failure!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm
    
    # wn = 1e12
    if abs(wo-w)/abs(w) < tol2:
    # if abs(wn-w) < tol2:
    # if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    # elif abs(wo-w)< tol2*1e-2:
    #     # print(abs(wo-w),abs(w),alpha,nrm,nrm0)
    #     print("stopped early due to stagnation | energy= %.10f" %w)
    #     # print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
    #     break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.01*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 2D mesh with ne= 52064 elements and nv= 26337 points and  104737 DOFs.
 
Iter:  1 | assem : 0.12s | CG took 0.37s with    0 iterations | alpha : 1.00 | energy = 0.0000000000 | relres = 1.00e+00 |
Iter:  2 | assem : 0.09s | CG took 0.35s with    0 iterations | alpha : 0.50 | energy = -2.8913137734 | relres = 8.37e-01 |
Iter:  3 | assem : 0.11s | CG took 0.38s with    0 iterations | alpha : 1.00 | energy = -3.8699596514 | relres = 2.71e-02 |
Iter:  4 | assem : 0.14s | CG took 0.42s with    0 iterations | alpha : 1.00 | energy = -3.9247930307 | relres = 1.99e-04 |
Iter:  5 | assem : 0.09s | CG took 0.37s with    0 iterations | alpha : 1.00 | energy = -3.9251954160 | relres = 1.43e-06 |
Iter:  6 | assem : 0.13s | CG took 0.38s with    0 iterations | alpha : 1.00 | energy = -3.9251984517 | relres = 4.28e-08 |
Iter:  7 | assem : 0.12s | CG took 0.36s with    0 iterations | alpha : 1.00 | energy = -3.9251985391 | relres = 3.81e-10 |
converged to desired tolerance


In [10]:
Draw(ngs.log(ngs.Norm(B)), mesh, min = -13, max = 0.5, settings={"Objects": {"Wireframe": False}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

In [11]:
# mask = ngs.GridFunction(H1)
# mask.Set(ngs.CF(1),ngs.BND)
# Draw(mask)

In [12]:
print(ngs.Integrate(ngs.Norm(B),mesh))
print(fun_W())

0.0029880601932106656
-3.925198539833347


In [13]:
scalars = mesh.MaterialCF({"coil_plus|coil_minus":1, "mid_steel|r_steel|l_steel": 2}, default = 0)
L2 = ngs.L2(mesh, order = 2)
scal = ngs.GridFunction(L2)
scal.Set(scalars)

vtk = ngs.VTKOutput(mesh, coefs = [B,ngs.curl(Hs),scal],names = ["B","J","scalars"], filename = "B3", order = 1)
vtk.Do()

'B3'

In [14]:
import pyvista as pv
pvmesh = pv.read("B3.vtu")
pvmesh.set_active_scalars("scalars")
pvmesh.set_active_vectors("B")
threshed = pvmesh.threshold([0.8,2.1])
threshed.set_active_vectors("B")

p = pv.Plotter()
p.add_mesh(threshed, style = 'surface', color = "w", opacity = 0.01, label = None)

arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01) #0.8
# arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01)
p.add_mesh(arrows, color = "yellow")

pvmesh.set_active_vectors("J")
arrows2 = pvmesh.glyph(scale = "J", orient = True, tolerance = 0.001, factor = 1*1e-8)
p.add_mesh(arrows2, color = "black")

p.show(jupyter_backend = "html")
# p.show()

ValueError: Data field (B) with type (FieldAssociation.POINT) could not be set as the active vectors

: 

In [ ]:
ngs.Integrate(J*A)

In [ ]:
ngs.Integrate(ngs.curl(Hs)*A-Hs*B, mesh)

In [ ]:
deg